##### Importing

The data set is too large do __NOT__ push it to online repository, either use path to read from your machine, or drag and drop into local repository but remember to remove it before pushing changes.

In [ ]:
import pandas as pd

db = pd.read_csv("Dataset name here", index_col=0)

db.shape

C:\Users\yesin\AppData\Local\Temp\ipykernel_40996\3787858537.py:3: DtypeWarning: Columns (42,166,167,168,169,174,175,178,179,182,183,188,189,190,191,194,195,203,204,205,218,219,220,231,232,233,238,240,241,249) have mixed types. Specify dtype option on import or set low_memory=False.
  db = pd.read_csv("NFL_09-18.csv", index_col=0)


(449371, 254)

##### Check all columns
_if you are running this using the entire dataset make sure to view as "scrollable" to see all features since its too large_

In [7]:
for col in db.columns:
    print(col)

game_id
home_team
away_team
posteam
posteam_type
defteam
side_of_field
yardline_100
game_date
quarter_seconds_remaining
half_seconds_remaining
game_seconds_remaining
game_half
quarter_end
drive
sp
qtr
down
goal_to_go
time
yrdln
ydstogo
ydsnet
desc
play_type
yards_gained
shotgun
no_huddle
qb_dropback
qb_kneel
qb_spike
qb_scramble
pass_length
pass_location
air_yards
yards_after_catch
run_location
run_gap
field_goal_result
kick_distance
extra_point_result
two_point_conv_result
home_timeouts_remaining
away_timeouts_remaining
timeout
timeout_team
td_team
posteam_timeouts_remaining
defteam_timeouts_remaining
total_home_score
total_away_score
posteam_score
defteam_score
score_differential
posteam_score_post
defteam_score_post
score_differential_post
no_score_prob
opp_fg_prob
opp_safety_prob
opp_td_prob
fg_prob
safety_prob
td_prob
extra_point_prob
two_point_conversion_prob
ep
epa
total_home_epa
total_away_epa
total_home_rush_epa
total_away_rush_epa
total_home_pass_epa
total_away_pass_epa
air_e

##### Check different play types

In [8]:
db["play_type"].value_counts(dropna=False)

play_type
pass           186677
run            132692
no_play         42431
kickoff         25552
punt            23914
NaN             12874
extra_point     10934
field_goal       9777
qb_kneel         3830
qb_spike          690
Name: count, dtype: int64

## Feature pruning 

__Removing__
- Metadata
- Player/team names 
- Post play_type data
- run/pass outcome

nfl_reducded = dataset - irrelevent features

In [12]:
relevant_cols = [
    "posteam", "defteam",
    "yardline_100", "qtr", "down", "ydstogo", "goal_to_go",
    "score_differential",
    "shotgun", "no_huddle",
    "quarter_seconds_remaining", "half_seconds_remaining", "game_seconds_remaining",
    "side_of_field", "home_team", "away_team", "play_type"
]


nfl_reduced = db[relevant_cols].copy()

In [13]:
nfl_reduced.head()

,posteam,defteam,yardline_100,qtr,down,ydstogo,goal_to_go,score_differential,shotgun,no_huddle,quarter_seconds_remaining,half_seconds_remaining,game_seconds_remaining,side_of_field,home_team,away_team,play_type
play_id,,,,,,,,,,,,,,,,,
46,PIT,TEN,30.0,1,NaN,0,0.0,NaN,0,0,900.0,1800.0,3600.0,TEN,PIT,TEN,kickoff
68,PIT,TEN,58.0,1,1.0,10,0.0,0.0,0,0,893.0,1793.0,3593.0,PIT,PIT,TEN,pass
92,PIT,TEN,53.0,1,2.0,5,0.0,0.0,0,0,856.0,1756.0,3556.0,PIT,PIT,TEN,run
113,PIT,TEN,56.0,1,3.0,8,0.0,0.0,1,0,815.0,1715.0,3515.0,PIT,PIT,TEN,pass
139,PIT,TEN,56.0,1,4.0,8,0.0,0.0,0,0,807.0,1707.0,3507.0,PIT,PIT,TEN,punt


## Play type pruning

__Removing__:
- no_play
- kickoff
- punt
- NaN
- extra_point
- field_goal
- qb_kneel
- qb_spike

nfl_pruned = play_type - irrelevent play types

In [36]:
relevant_play_types = ["run", "pass"]
nfl_pruned = nfl_reduced[nfl_reduced["play_type"].isin(relevant_play_types)].copy()

In [37]:
nfl_pruned["play_type"].value_counts(dropna=False)

play_type
pass    186677
run     132692
Name: count, dtype: int64

## Data Type Encoding & Mapping

Transforming data type to either numeric or boolean 

##### Check pruned dataset data types

In [24]:
nfl_pruned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 319369 entries, 68 to 4106
Data columns (total 17 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   posteam                    319369 non-null  object 
 1   defteam                    319369 non-null  object 
 2   yardline_100               319369 non-null  float64
 3   qtr                        319369 non-null  int64  
 4   down                       318691 non-null  float64
 5   ydstogo                    319369 non-null  int64  
 6   goal_to_go                 319369 non-null  float64
 7   score_differential         319369 non-null  float64
 8   shotgun                    319369 non-null  int64  
 9   no_huddle                  319369 non-null  int64  
 10  quarter_seconds_remaining  319369 non-null  float64
 11  half_seconds_remaining     319352 non-null  float64
 12  game_seconds_remaining     319355 non-null  float64
 13  side_of_field              319369 n

## Mapping play_type

Since play_type has been pruned and only contains run/pass, I am going to encode it were pass = 1 and run = 0

In [ ]:
nfl_pruned["play_type"] = nfl_pruned["play_type"].map({"pass": 1, "run": 0})

In [45]:
nfl_pruned["play_type"].head(5)

play_id
68     1
92     0
113    1
162    0
183    1
Name: play_type, dtype: int64

## One-Hot Encoding for obj dtypes

Cols to be encoded:
- posteam
- defteam
- side_of_field
- home_team
- away_team

In [46]:
nfl_pruned.dtypes

posteam                       object
defteam                       object
yardline_100                 float64
qtr                            int64
down                         float64
ydstogo                        int64
goal_to_go                   float64
score_differential           float64
shotgun                        int64
no_huddle                      int64
quarter_seconds_remaining    float64
half_seconds_remaining       float64
game_seconds_remaining       float64
side_of_field                 object
home_team                     object
away_team                     object
play_type                      int64
dtype: object

__Seperate play_type from dataset to avoid encoding it__

In [47]:
y = nfl_pruned["play_type"]
X = nfl_pruned.drop(columns=["play_type"])

In [48]:
cols_to_encode = ["posteam", "defteam", "side_of_field", "home_team", "away_team"]

nfl_encoded = pd.get_dummies(X, columns=cols_to_encode, drop_first=True)

In [49]:
nfl_encoded.info()

<class 'pandas.core.frame.DataFrame'>
Index: 319369 entries, 68 to 4106
Columns: 182 entries, yardline_100 to away_team_WAS
dtypes: bool(171), float64(7), int64(4)
memory usage: 81.3 MB


__Concat play_type with encoded dataset into one clean data set__

In [50]:
nfl_cleaned = pd.concat([nfl_encoded, y], axis=1)

__Write to a new csv file__

In [52]:
nfl_cleaned.to_csv("nfl_cleaned.csv", index=False)